# NLP con Deep Learning: Transformers

# - DistilBert

In [3]:
import torch
import numpy as np
import pandas as pd

In [4]:
device = "cuda:0" if torch.cuda.is_available() else "cpu" #para gpu
print(device)

cuda:0


In [5]:
df = pd.read_csv('dataset_final_actualizacion_mayus2.csv')
df

Periódico                                       Hipervínculo  \
0       ElDiario.es  https://www.eldiario.es/politica/pp-recupera-l...   
1       ElDiario.es  https://www.eldiario.es/politica/58-ciudadanos...   
2       ElDiario.es  https://www.eldiario.es/politica/diputado-ayus...   
3       ElDiario.es  https://www.eldiario.es/andalucia/almeria/psoe...   
4       ElDiario.es  https://www.eldiario.es/castilla-la-mancha/pol...   
...             ...                                                ...   
4974  HayNoticia.es  https://haynoticia.es/una-empresa-catalana-tri...   
4975  HayNoticia.es  https://haynoticia.es/hospitalizado-tras-inten...   
4976  HayNoticia.es  https://haynoticia.es/lleva-300-bolsas-de-plas...   
4977  HayNoticia.es  https://haynoticia.es/la-dgt-multara-a-los-con...   
4978  HayNoticia.es  https://haynoticia.es/una-sentencia-prohibe-a-...   

     Fecha publicación                                            Titular  \
0           2023-07-11  El PP se recupera ligeramente pero sigue sin o...   
1           2023-07-11  Un 58% de los ciudadanos critican los pactos c...   
2           2023-07-11  Un diputado de Ayuso reúne a un grupo de vícti...   
3           2023-07-11  El PSOE de Mójacar recurre ante el Constitucio...   
4           2023-07-11  La exalcaldesa de Toledo pide a Feijóo que fir...   
...                ...                                                ...   
4974        2018-11-06  Una empresa catalana triunfa con su papel higi...   
4975        2018-10-31  Hospitalizado tras intentar sacarse una muela ...   
4976        2018-10-30  Lleva 300 bolsas de plástico a Mercadona para ...   
4977        2018-10-28  La DGT multará a los conductores que no hayan ...   
4978        2018-10-25  Una Sentencia prohibe a los comercios poner vi...   

                                              Subtítulo  \
0     La estimación de escaños no se mueve en la enc...   
1     El 17,8% de quienes apuestan por Feijóo ven “m...   
2     Daniel Portero trata con urgencia de contrarre...   
3     Ya con nuevo alcalde del PP, los socialistas r...   
4     La socialista, también número dos al Congreso,...   
...                                                 ...   
4974                                                NaN   
4975                                                NaN   
4976                                                NaN   
4977                                                NaN   
4978                                                NaN   

                                                 Cuerpo           Categoría  \
0     El partido popular se recupera muy ligeramente...            politica   
1     El PP ha necesitado a Vox para recuperar buena...            politica   
2     El diputado del PP en la Asamblea de Madrid Da...            politica   
3     Mojácar retomó el ritmo político con la procla...           andalucia   
4     La exalcaldesa de Toledo y 'número dos' de la ...  castilla-la-mancha   
...                                                 ...                 ...   
4974  La empresa catalana Banys Nous de Barcelona es...        curiosidades   
4975  Sucedió el pasado lunes poco después de las 20...        curiosidades   
4976  Un joven ha sido noticia ayer tras presentarse...        curiosidades   
4977  Este domingo nos ha tocado hacer por segunda v...        curiosidades   
4978  Así se ha pronunciado la Audiencia Provincial ...        curiosidades   

      Target  
0          0  
1          0  
2          0  
3          0  
4          0  
...      ...  
4974       1  
4975       1  
4976       1  
4977       1  
4978       1  

[4979 rows x 8 columns]

In [6]:
df.Target.value_counts()

0    2506
1    2473
Name: Target, dtype: int64

In [7]:
df["Texto"] = df.Titular + " " + df.Cuerpo
df.head()

Periódico                                       Hipervínculo  \
0  ElDiario.es  https://www.eldiario.es/politica/pp-recupera-l...   
1  ElDiario.es  https://www.eldiario.es/politica/58-ciudadanos...   
2  ElDiario.es  https://www.eldiario.es/politica/diputado-ayus...   
3  ElDiario.es  https://www.eldiario.es/andalucia/almeria/psoe...   
4  ElDiario.es  https://www.eldiario.es/castilla-la-mancha/pol...   

  Fecha publicación                                            Titular  \
0        2023-07-11  El PP se recupera ligeramente pero sigue sin o...   
1        2023-07-11  Un 58% de los ciudadanos critican los pactos c...   
2        2023-07-11  Un diputado de Ayuso reúne a un grupo de vícti...   
3        2023-07-11  El PSOE de Mójacar recurre ante el Constitucio...   
4        2023-07-11  La exalcaldesa de Toledo pide a Feijóo que fir...   

                                           Subtítulo  \
0  La estimación de escaños no se mueve en la enc...   
1  El 17,8% de quienes apuestan por Feijóo ven “m...   
2  Daniel Portero trata con urgencia de contrarre...   
3  Ya con nuevo alcalde del PP, los socialistas r...   
4  La socialista, también número dos al Congreso,...   

                                              Cuerpo           Categoría  \
0  El partido popular se recupera muy ligeramente...            politica   
1  El PP ha necesitado a Vox para recuperar buena...            politica   
2  El diputado del PP en la Asamblea de Madrid Da...            politica   
3  Mojácar retomó el ritmo político con la procla...           andalucia   
4  La exalcaldesa de Toledo y 'número dos' de la ...  castilla-la-mancha   

   Target                                              Texto  
0       0  El PP se recupera ligeramente pero sigue sin o...  
1       0  Un 58% de los ciudadanos critican los pactos c...  
2       0  Un diputado de Ayuso reúne a un grupo de vícti...  
3       0  El PSOE de Mójacar recurre ante el Constitucio...  
4       0  La exalcaldesa de Toledo pide a Feijóo que fir...

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df["Texto"].tolist(), 
                                                  df["Target"].tolist(), 
                                                  test_size=0.45, 
                                                  stratify=df["Target"],
                                                  random_state=42)

# HF Transformers

In [9]:
!pip install transformers[torch] --quiet
!pip install accelerate -U --quiet

In [10]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [11]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

In [12]:
type(train_encodings)

transformers.tokenization_utils_base.BatchEncoding

In [13]:
train_encodings.keys()

dict_keys(['input_ids', 'attention_mask'])

In [14]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        #item es un diccionario que contiene tres keys: 'input_ids', 'attention_mask' y 'labels'
        #cada key contiene el tensor correspodiente al indice idx
        #item = {key: torch.tensor(val[idx]).to(device) for key, val in self.encodings.items()}
        item = {}
        item['input_ids'] = torch.tensor(self.encodings['input_ids'][idx]).to(device)
        item['attention_mask'] = torch.tensor(self.encodings['attention_mask'][idx]).to(device)
        item['labels'] = torch.tensor(self.labels[idx]).to(device)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = MyDataset(train_encodings, y_train)
test_dataset = MyDataset(test_encodings, y_test)

In [15]:
print(len(train_dataset),len(test_dataset))

2738 2241


In [16]:
train_dataset[0]

{'input_ids': tensor([  101,  2474, 14099,  7367,  8566, 14693,  2239,  2139,  5631,  1024,
         10384,  2229,  1010, 20356, 17540,  2890, 11297,  2080,  1061, 27218,
          2891, 27709,  2139,  3565,  4605,  9594,  2080, 10861, 11265,  3995,
          7405,  2099,  8254,  4078, 26642,  2080, 22959,  2618,  2033,  8583,
         11498,  9530,  3366, 25698,  2099,  3050,  2566, 15630,  2891,  2139,
          2474, 20759,  1010,  3449,  9530, 14697,  2080,  1061,  3449, 18280,
         24996,  2080,  2139,  3665,  2063,  2139,  3516,  1012,  9594,  2080,
         10861,  4078, 10010,  7559,  2474,  6728, 10446,  2139,  5631,  3509,
          1061,  2139,  5869,  2655,  2229,  3972, 18120, 11498,  3902, 10010,
         20639,  3597,  4372,  2474,  1062,  7856,  3972,  2524,  2600,  3346,
          1010,  3449, 27441,  3366,  2080,  2123,  3207,  5828,  2632, 10010,
         10936,  2632,  6844,  5292,  3401, 14477,  2015,  7367, 24805,  2015,
         10514,  3539,  2099, 14841,  8

In [17]:
# !pip install evaluate --quiet

In [18]:
import evaluate
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [21]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=6,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    dataloader_pin_memory=False,     # remove if possible for faster training
    evaluation_strategy = "epoch",
    output_dir="./results"
)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2).to(device)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,            # evaluation dataset
    compute_metrics=compute_metrics,
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'classifier.

In [22]:
trainer.train()

C:\Users\Usuario\anaconda3\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


TrainOutput(global_step=457, training_loss=0.3772920904848195, metrics={'train_runtime': 198.8864, 'train_samples_per_second': 13.767, 'train_steps_per_second': 2.298, 'total_flos': 362695737520128.0, 'train_loss': 0.3772920904848195, 'epoch': 1.0})

In [23]:
trainer.evaluate()

{'eval_loss': 0.15333150327205658,
 'eval_accuracy': 0.9589468987059349,
 'eval_runtime': 39.098,
 'eval_samples_per_second': 57.318,
 'eval_steps_per_second': 7.187,
 'epoch': 1.0}